$$-\Delta u = 100 \sin(\pi x)$$ in $\Omega$ with $u = 0$ on $\partial \Omega$

In [1]:
import numpy as np
from skfem import LinearForm
@LinearForm
def loading(v, w):
    return(100*np.sin(np.pi * w.x[0]) * v)

In [2]:
from skfem import *
from skfem.models.poisson import *
import numpy as np
from scipy.sparse import spmatrix
from scipy.sparse.linalg import LinearOperator

p = np.linspace(0, 1, 16)
m = MeshTet.init_tensor(*(p,) * 3)

e = ElementTetP1()
basis = InteriorBasis(m, e)

A = asm(laplace, basis)
b = asm(loading, basis)

I = m.interior_nodes()

x = 0. * b

if __name__ == "__main__":
    verbose = True
else:
    verbose = False

Aint, bint = condense(A, b, I=I, expand=False)

preconditioners = [None, build_pc_ilu(Aint, drop_tol=1e-3)]

try:
    from pyamg import smoothed_aggregation_solver

    def build_pc_amgsa(A: spmatrix, **kwargs) -> LinearOperator:
        """AMG (smoothed aggregation) preconditioner"""
        return smoothed_aggregation_solver(A, **kwargs).aspreconditioner()

    preconditioners.append(build_pc_amgsa(Aint))

except ImportError:
    print('Skipping PyAMG')

try:
    import pyamgcl

    def build_pc_amgcl(A: spmatrix, **kwargs) -> LinearOperator:
        """AMG preconditioner"""

        if hasattr(pyamgcl, 'amgcl'):  # v. 1.3.99+
            return pyamgcl.amgcl(A, **kwargs)
        else:
            return pyamgcl.amg(A, **kwargs)

    preconditioners.append(build_pc_amgcl(Aint))

except ImportError:
    print('Skipping pyamgcl')

for pc in preconditioners:
    x[I] = solve(Aint, bint, solver=solver_iter_pcg(verbose=verbose, M=pc))


if verbose:
    from os.path import splitext
    from sys import argv

    m.save(splitext(argv[0])[0] + ".vtk", {'potential': x})

Skipping PyAMG
Skipping pyamgcl
51.53943016037791
79.58124646858306
94.11287254418697
101.92455617160694
107.23760227412303
110.08851830474401
111.69549272892144
112.36154806359076
112.57482813548702
112.61341668225431
112.62182126040526
112.6261105325878
112.6273894977482
112.62770040700805
112.62778466087978
112.62779881133666
112.62780154504179
112.62780179913209
112.62780181794989
cg converged to tol=default and atol=default
111.71252387758108
112.7862586662018
112.64502452791247
112.62811729326103
112.62769118718157
cg converged to tol=default and atol=default
